In [4]:
# CVANN - Computer Vision Artificial Neural Network

import os
import numpy as np

from keras.models import Sequential
from keras.layers import MaxPooling2D, Convolution2D
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.utils.np_utils import to_categorical

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [5]:
def sep_grayscale_intervals(img, num_intervals=4, output_path=None):

    img_array = img_to_array(img)

    for img in range(len(img_array)):
        for i in range(len(img_array[img])):
            for j in range(len(img_array[img][i])):

                pixel = img_array[img][i][j]
                new_value = pixel

                for interval in range(num_intervals):
                    interval_max = 255-((256/num_intervals)*interval)
                    if pixel < interval_max:
                        new_value = interval_max-(256/num_intervals)

                img_array[img][i][j] = new_value

    if output_path:
        img.save(output_path, 'JPEG')

    return array_to_img(img_array)

In [6]:
def load_data(img_dir_name, img_dimensions=(120, 90)):

    print('\nLoading data: "' + str(img_dir_name) + '"')

    directory = '..' + os.sep + 'resources' + os.sep + str(img_dir_name)
    sub_dir_names = [file for file in os.listdir(directory) if not file.startswith('.')]

    # calculate the total number of images | initialize the dictionary mapping the names to integers
    total_num_images = 0
    i = 0
    dictionary = {}

    for sub_dir in sub_dir_names:
        sub_dir_path = directory + os.sep + sub_dir
        image_files = [file for file in os.listdir(sub_dir_path) if not file.startswith('.')]
        total_num_images += len(image_files)

        dictionary[sub_dir] = i
        i += 1

    print('Loading ' + str(total_num_images) + ' images\n')

    data = []
    labels = np.zeros((total_num_images, 1)).astype('int')
    i = 0
    for sub_dir in sub_dir_names:
        print('current directory of images: ' + sub_dir)

        label = int(dictionary[sub_dir])  # the 'label' associated to this folder
        sub_dir_path = directory + os.sep + sub_dir
        img_files = [file for file in os.listdir(sub_dir_path) if not file.startswith('.')]

        for file in img_files:
            file_path = sub_dir_path + os.sep + file
            img = load_img(file_path, grayscale=True, target_size=img_dimensions)
            # img = sep_grayscale_intervals(img, num_intervals=8)  # testing phase
            img_array = img_to_array(img)

            labels = np.insert(labels, i, label, axis=0)  # insert the label at position i
            labels = np.delete(labels, -1, axis=0)  # remove the one too many element
            data.append(img_array)
            i += 1

    data = np.asarray(data)
    data /= 255
    nb_classes = len(dictionary)
    labels = to_categorical(labels, nb_classes=nb_classes)

    return data, labels, nb_classes

In [7]:
def neural_network(train, validation, nb_epoch=5, batch_size=40, activations=('relu', 'relu', 'softmax'),
                   nb_filters=30, img_dimensions=(120, 90), pool_dimensions=(8, 6), conv_dimensions=(12, 9),
                   optimizer='rmsprop'):

    model = Sequential()
    model.add(Convolution2D(nb_filter=nb_filters, nb_row=conv_dimensions[0], nb_col=conv_dimensions[1],
                            input_shape=(1, img_dimensions[0], img_dimensions[1]), activation=activations[0]))

    model.add(MaxPooling2D(pool_size=pool_dimensions))
    model.add(Dropout(0.5))
    model.add(Flatten())  # converts our 2D feature maps to 1D feature vectors
    model.add(Dense(128, activation=activations[1]))
    model.add(Dropout(0.3))
    model.add(Dense(train[2], activation=activations[2]))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(x=train[0], y=train[1], nb_epoch=nb_epoch, batch_size=batch_size)

    print('\n')
    loss, accuracy = model.evaluate(x=validation[0], y=validation[1], batch_size=batch_size, verbose=1)
    print('loss: ' + str(round(loss, 4)))
    print('accuracy: ' + str(round(accuracy, 4)) + '\n')

    return model

In [70]:
train = load_data('AV_train')
validation = load_data('AV_validation')


Loading data: "CalTech_101"
Loading 9145 images

current directory of images: accordion
current directory of images: airplanes
current directory of images: anchor
current directory of images: ant
current directory of images: BACKGROUND_Google
current directory of images: barrel
current directory of images: bass
current directory of images: beaver
current directory of images: binocular
current directory of images: bonsai
current directory of images: brain
current directory of images: brontosaurus
current directory of images: buddha
current directory of images: butterfly
current directory of images: camera
current directory of images: cannon
current directory of images: car_side
current directory of images: ceiling_fan
current directory of images: cellphone
current directory of images: chair
current directory of images: chandelier
current directory of images: cougar_body
current directory of images: cougar_face
current directory of images: crab
current directory of images: crayfish
curr

In [71]:
model = neural_network(train, validation, activations=('relu', 'relu', 'softmax'), nb_epoch=1, batch_size=30, optimizer='rmsprop')

Epoch 1/1
9145/9145 [==============================] - 454s - loss: 3.6470 - acc: 0.2504   


9145/9145 [==============================] - 244s   
loss: 2.9359
accuracy: 0.3872



In [73]:
model.fit(x=train[0], y=train[1], nb_epoch=4, batch_size=10)
print('\n')
loss, accuracy = model.evaluate(x=validation[0], y=validation[1], batch_size=1, verbose=1)
print('loss: ' + str(round(loss, 4)))
print('accuracy: ' + str(round(accuracy, 4)) + '\n')

Epoch 1/4
9145/9145 [==============================] - 474s - loss: 2.3606 - acc: 0.4742   
Epoch 2/4
9145/9145 [==============================] - 435s - loss: 2.2913 - acc: 0.4866   
Epoch 3/4
9145/9145 [==============================] - 436s - loss: 2.2137 - acc: 0.4975   
Epoch 4/4
9145/9145 [==============================] - 434s - loss: 2.1935 - acc: 0.5088   


9145/9145 [==============================] - 273s   
loss: 1.7076
accuracy: 0.6232

